# DW_pandas_Task6

## 20201228

DataWhale_pandas组队学习第六章，连接，加油！！！

### 关系型连接

In [1]:
import pandas as pd
import numpy as np

在关系型连接中，$\color{red}{键}$是十分重要的，往往用`on`参数表示。在`pandas`中的关系型连接函数`merge`和`join`中提供了`how`参数来代表连接形式，分为左连接`left`、右连接`right`、内连接`inner`、外连接`outer`，所谓左连接即以左边的键为准，如果右边表中的键于左边存在，那么就添加到左边，否则则处理为缺失值，右连接类似处理。内连接只负责合并两边同时出现的键，而外连接则会在内连接的基础上包含只在左边出现以及只在右边出现的值，因此外连接又叫全连接。

同一个表中的键没有出现重复的情况，那么如果出现重复的键应该如何处理？只需把握一个原则，即只要两边同时出现的值，就以笛卡尔积的方式加入，如果单边出现则根据连接形式进行处理。其中，关于笛卡尔积可用如下例子说明：设左表中键`张三`出现两次，右表中的`张三`也出现两次，那么逐个进行匹配，最后产生的表必然包含`2*2`个姓名为`张三`的行。
显然在不同的场合应该使用不同的连接形式。其中左连接和右连接是等价的，由于它们的结果中的键是被一侧的表确定的，因此常常用于有方向性地添加到目标表。内外连接两侧的表，经常是地位类似的，想取出键的交集或者并集，具体的操作还需要业务的需求来判断。


In [2]:
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'], 'Age':[20,30]})
df2 = pd.DataFrame({'Name':['Si Li','Wu Wang'], 'Gender':['F','M']})
df1.merge(df2, on='Name', how='left')

,Name,Age,Gender
0,San Zhang,20,NaN
1,Si Li,30,F


In [3]:
df1.merge(df2, on='Name', how='outer')

,Name,Age,Gender
0,San Zhang,20.0,NaN
1,Si Li,30.0,F
2,Wu Wang,NaN,M


In [4]:
df1 = pd.DataFrame({'Name':['San Zhang'],'Grade':[70],'Test':[77]})
df2 = pd.DataFrame({'Name':['San Zhang'],'Grade':[80],'Test':[88]})
df1.merge(df2, on='Name', how='left', suffixes=['_Chinese','_Math'])

,Name,Grade_Chinese,Test_Chinese,Grade_Math,Test_Math
0,San Zhang,70,77,80,88


suffixes对所有重复的列都有效，为列名添加后缀。

In [5]:
df1 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],
                    'Age':[20, 21],
                    'Class':['one', 'two']})
df2 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],
                    'Gender':['F', 'M'],
                    'Class':['two', 'one']})
df1

,Name,Age,Class
0,San Zhang,20,one
1,San Zhang,21,two


In [6]:
df2

,Name,Gender,Class
0,San Zhang,F,two
1,San Zhang,M,one


In [7]:
df1.merge(df2, on='Name', how='outer')

,Name,Age,Class_x,Gender,Class_y
0,San Zhang,20,one,F,two
1,San Zhang,20,one,M,one
2,San Zhang,21,two,F,two
3,San Zhang,21,two,M,one


姓名都是重复的，采用外部链接，结果把所有的数据都匹配，形成错误的表格。

In [8]:
df1.merge(df2, on=['Name','Class'], how='outer')

,Name,Age,Class,Gender
0,San Zhang,20,one,M
1,San Zhang,21,two,F


从上面的例子来看，在进行基于唯一性的连接下，如果键不是唯一的，那么结果就会产生问题。举例中的行数很少，但如果实际数据中有几十万到上百万行的进行合并时，如果想要保证唯一性，除了用`duplicated`检查是否重复外，`merge`中也提供了`validate`参数来检查连接的唯一性模式。这里共有三种模式，即一对一连接`1:1`，一对多连接`1:m`，多对一连接`m:1`连接，第一个是指左右表的键都是唯一的，后面两个分别指左表键唯一和右表键唯一。

In [9]:
pd.merge(df1, df2, on='Class', how='outer', validate='one_to_many')

,Name_x,Age,Class,Name_y,Gender
0,San Zhang,20,one,San Zhang,M
1,San Zhang,21,two,San Zhang,F


validate：string，默认无。如果指定，则检查merge是否为指定类型。

“one_to_one”或“1：1”：检查合并键是否在左右数据集中都是唯一的。

“one_to_many”或“1：m”：检查合并键是否在左数据集中是唯一的。

“many_to_one”或“m：1”：检查合并键是否在右侧数据集中是唯一的。

“many_to_many”或“m：m”：允许，但不会导致检查。

对列Name采用除m:m外的都出错，在实际运用中这样检查还是比较繁琐的。

#### 【练一练】
上面以多列为键的例子中，错误写法显然是一种多对多连接，而正确写法是一对一连接，请修改原表，使得以多列为键的正确写法能够通过`validate='1:m'`的检验，但不能通过`validate='m:1'`的检验。

In [10]:
df1.merge(df2, on=['Name','Class'], how='outer', validate='1:m')

,Name,Age,Class,Gender
0,San Zhang,20,one,M
1,San Zhang,21,two,F


In [11]:
df1.merge(df2, on=['Name','Class'], how='outer', validate='m:1')

,Name,Age,Class,Gender
0,San Zhang,20,one,M
1,San Zhang,21,two,F


In [12]:
df1 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],
                    'Age':[20, 21],
                    'Class':['one', 'two']})
df2 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],
                    'Gender':['F', 'M'],
                    'Class':['two', 'two']})
df1.merge(df2, on=['Name','Class'], how='outer', validate='1:m')

,Name,Age,Class,Gender
0,San Zhang,20,one,NaN
1,San Zhang,21,two,F
2,San Zhang,21,two,M


In [13]:
df1.merge(df2, on=['Name','Class'], how='outer', validate='m:1')

MergeError: Merge keys are not unique in right dataset; not a many-to-one merge

m:1检查合并键是否在右侧数据集中是唯一的，修改右表Class为重复值，则无法通过。

In [14]:
df1 = pd.DataFrame({'Grade':[70]}, index=pd.Series(['San Zhang'], name='Name'))
df2 = pd.DataFrame({'Grade':[80]}, index=pd.Series(['San Zhang'], name='Name'))
df1.join(df2, how='left', lsuffix='_Chinese', rsuffix='_Math')

,Grade_Chinese,Grade_Math
Name,,
San Zhang,70,80


In [15]:
df1 = pd.DataFrame({'Age':[20,21]}, index=pd.MultiIndex.from_arrays([['San Zhang', 'San Zhang'],['one', 'two']], names=('Name','Class')))
df2 = pd.DataFrame({'Gender':['F', 'M']}, index=pd.MultiIndex.from_arrays([['San Zhang', 'San Zhang'],['two', 'one']], names=('Name','Class')))
df1

Age
Name      Class     
San Zhang one     20
          two     21

In [16]:
df2

Gender
Name      Class       
San Zhang two        F
          one        M

In [17]:
df1.join(df2, on=['Name','Class'])

Age Gender
Name      Class            
San Zhang one     20      M
          two     21      F

索引连接，就是把索引当作键，因此这和值连接本质上没有区别，`pandas`中利用`join`函数来处理索引连接，它的参数选择要少于`merge`，除了必须的`on`和`how`之外，可以对重复的列指定左右后缀`lsuffix`和`rsuffix`。其中，`on`参数指索引名，单层索引时省略参数表示按照当前索引连接。

单层索引默认是所有索引，多层则用on指定索引名。

### 方向连接

In [18]:
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'], 'Age':[20,30]})
df2 = pd.DataFrame({'Name':['Wu Wang'], 'Age':[40]})
pd.concat([df1, df2])

,Name,Age
0,San Zhang,20
1,Si Li,30
0,Wu Wang,40


In [19]:
df2 = pd.DataFrame({'Grade':[80, 90]})
df3 = pd.DataFrame({'Gender':['M', 'F']})
pd.concat([df1, df2, df3], 1)

,Name,Age,Grade,Gender
0,San Zhang,20,80,M
1,Si Li,30,90,F


在`concat`中，最常用的有三个参数，它们是`axis, join, keys`，分别表示拼接方向，连接形式，以及在新表中指示来自于哪一张旧表的名字。在默认状态下的`axis=0`，表示纵向拼接多个表，常常用于多个样本的拼接；而`axis=1`表示横向拼接多个表，常用于多个字段或特征的拼接。

纵向拼接指向下拼接⬇，横向拼接指向右拼接➡，这样更方便理解。

In [20]:
df2 = pd.DataFrame({'Grade':[80, 90]}, index=[1, 2])
pd.concat([df1, df2], 1)

,Name,Age,Grade
0,San Zhang,20.0,NaN
1,Si Li,30.0,80.0
2,NaN,NaN,90.0


In [21]:
pd.concat([df1, df2], axis=1, join='inner')

,Name,Age,Grade
1,Si Li,30,80


concat是处理关系型合并的函数，但是它仍然是关于索引进行连接的。纵向拼接会根据列索引对其，默认状态下join=outer，表示保留所有的列，并将不存在的值设为缺失；join=inner，表示保留两个表都出现过的列。横向拼接则根据行索引对齐，join参数可以类似设置。

In [22]:
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'], 'Age':[20,21]})
df2 = pd.DataFrame({'Name':['Wu Wang'],'Age':[21]})
pd.concat([df1, df2], keys=['one', 'two'])

Name  Age
one 0  San Zhang   20
    1      Si Li   21
two 0    Wu Wang   21

keys参数的使用场景在于多个表合并后，用户仍然想要知道新表中的数据来自于哪个原表，这时可以通过keys参数产生多级索引进行标记。

In [23]:
s = pd.Series(['Wu Wang', 21], index = df1.columns)
df1.append(s, ignore_index=True)

,Name,Age
0,San Zhang,20
1,Si Li,21
2,Wu Wang,21


In [24]:
s = pd.Series([80, 90])
df1.assign(Grade=s)

,Name,Age,Grade
0,San Zhang,20,80
1,Si Li,21,90


In [25]:
df1['Grade'] = s
df1

,Name,Age,Grade
0,San Zhang,20,80
1,Si Li,21,90


把一个序列追加到表的行末或者列末，则可以分别使用append和assign方法。对于assign而言，虽然可以利用其添加新的列，但一般通过df['new_col'] = ...的形式就可以等价地添加新列。同时，使用[]修改的缺点是它会直接在原表上进行改动，而assign返回的是一个临时副本。

### 类连接操作

In [26]:
df1 = pd.DataFrame({'Name':['San Zhang', 'Si Li', 'Wu Wang'],
                    'Age':[20, 21 ,21],
                    'Class':['one', 'two', 'three']})
df2 = pd.DataFrame({'Name':['San Zhang', 'Li Si', 'Wu Wang'],
                    'Age':[20, 21 ,21],
                    'Class':['one', 'two', 'Three']})
df1.compare(df2)

Name         Class       
    self  other   self  other
1  Si Li  Li Si    NaN    NaN
2    NaN    NaN  three  Three

In [27]:
df1.compare(df2, keep_shape=True).count()

Name   self     1
       other    1
Age    self     0
       other    0
Class  self     1
       other    1
dtype: int64

compare是在1.1.0后引入的新函数，它能够比较两个表或者序列的不同处并将其汇总展示,结果中返回了不同值所在的行列，如果相同则会被填充为缺失值NaN，其中other和self分别指代传入的参数表和被调用的表自身。

如果想要完整显示表中所有元素的比较情况，可以设置keep_shape=True

In [28]:
def choose_min(s1, s2):
    s2 = s2.reindex_like(s1)  # 返回匹配s1的对象 
    res = s1.where(s1<s2, s2) # 若s1<s2，则替换为s2
    res = res.mask(s1.isna()) # isna表示是否为缺失值，返回布尔序列
    return res
df1 = pd.DataFrame({'A':[1,2], 'B':[3,4], 'C':[5,6]})
df2 = pd.DataFrame({'B':[5,6], 'C':[7,8], 'D':[9,10]}, index=[1,2])
df1.combine(df2, choose_min)

,A,B,C,D
0,NaN,NaN,NaN,NaN
1,NaN,4.0,6.0,NaN
2,NaN,NaN,NaN,NaN


`combine`函数能够让两张表按照一定的规则进行组合，在进行规则比较时会自动进行列索引的对齐。对于传入的函数而言，每一次操作中输入的参数是来自两个表的同名`Series`，依次传入的列是两个表列名的并集。

#### 【练一练】
请在上述代码的基础上修改，保留`df2`中4个未被`df1`替换的相应位置原始值。

In [29]:
def choose_min(s1, s2):
    s2 = s2.reindex_like(s1)  # 返回匹配s1的对象
    res = s1.where(s1<s2, s2) # 若s1<s2，则替换为s2
    res = res.mask(s2.isna()) # isna表示是否为缺失值，返回布尔序列
    return res
df1 = pd.DataFrame({'A':[1,2], 'B':[3,4], 'C':[5,6]})
df2 = pd.DataFrame({'B':[5,6], 'C':[7,8], 'D':[9,10]}, index=[1,2])
df1.combine(df2, choose_min)

,A,B,C,D
0,NaN,NaN,NaN,NaN
1,NaN,4.0,6.0,9.0
2,NaN,6.0,8.0,10.0


是这个意思吗？模模糊糊的

In [30]:
df1.combine(df2, choose_min, overwrite=False)

,A,B,C,D
0,1.0,NaN,NaN,NaN
1,2.0,4.0,6.0,9.0
2,NaN,6.0,8.0,10.0


设置`overtwrite`参数为`False`可以保留$\color{red}{被调用表}$中未出现在传入的参数表中的列，而不会设置未缺失值

#### 【练一练】
除了`combine`之外，`pandas`中还有一个`combine_first`方法，其功能是在对两张表组合时，若第二张表中的值在第一张表中对应索引位置的值不是缺失状态，那么就使用第一张表的值填充。下面给出一个例子，请用`combine`函数完成相同的功能。

In [31]:
df1 = pd.DataFrame({'A':[1,2], 'B':[3,np.nan]})
df2 = pd.DataFrame({'A':[5,6], 'B':[7,8]}, index=[1,2])
df1.combine_first(df2)

,A,B
0,1.0,3.0
1,2.0,7.0
2,6.0,8.0


In [32]:
def choose_min(s1, s2):
    res = s1.where(s1<s2, s2)
    res = res.mask(s2.isna(), s1)
    return res
df1.combine(df2, choose_min)

,A,B
0,1.0,3.0
1,2.0,7.0
2,6.0,8.0


学过就忘，重新翻回第二章复习，总算折腾出来。

## 练习
### Ex1：美国疫情数据集

现有美国4月12日至11月16日的疫情报表，请将`New York`的`Confirmed, Deaths, Recovered, Active`合并为一张表，索引为按如下方法生成的日期字符串序列：

In [33]:
date = pd.date_range('20200412', '20201116').to_series()
date = date.dt.month.astype('string').str.zfill(2) +'-'+ date.dt.day.astype('string').str.zfill(2) +'-'+ '2020'
date = date.tolist()
date[:5]

['04-12-2020', '04-13-2020', '04-14-2020', '04-15-2020', '04-16-2020']

In [34]:
usa = pd.read_csv('../data/us_report/{}.csv'.format(date[2]))
usa.head()

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Testing_Rate,Hospitalization_Rate
0,Alabama,US,2020-04-14 23:33:31,32.3182,-86.9023,3953,114,NaN,3839.0,1.0,84.305541,33117.0,493.0,2.883886,84000001,USA,706.285508,12.471541
1,Alaska,US,2020-04-14 23:33:31,61.3707,-152.4044,285,9,98.0,276.0,2.0,47.678875,8348.0,32.0,3.157895,84000002,USA,1396.572809,11.228070
2,American Samoa,US,NaN,-14.2710,-170.1320,0,0,NaN,NaN,60.0,0.000000,3.0,NaN,NaN,16,ASM,5.391708,NaN
3,Arizona,US,2020-04-14 23:33:31,33.7298,-111.4312,3809,131,249.0,3678.0,4.0,52.330651,44096.0,580.0,3.439223,84000004,USA,605.821053,15.227094
4,Arkansas,US,2020-04-14 23:33:31,34.9697,-92.3731,1498,32,427.0,1466.0,5.0,57.859574,21131.0,130.0,2.136182,84000005,USA,816.175343,8.678238


In [35]:
usa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Province_State        59 non-null     object 
 1   Country_Region        59 non-null     object 
 2   Last_Update           57 non-null     object 
 3   Lat                   56 non-null     float64
 4   Long_                 56 non-null     float64
 5   Confirmed             59 non-null     int64  
 6   Deaths                59 non-null     int64  
 7   Recovered             32 non-null     float64
 8   Active                58 non-null     float64
 9   FIPS                  58 non-null     float64
 10  Incident_Rate         56 non-null     float64
 11  People_Tested         56 non-null     float64
 12  People_Hospitalized   49 non-null     float64
 13  Mortality_Rate        57 non-null     float64
 14  UID                   59 non-null     int64  
 15  ISO3                  59 

In [36]:
usa_demo = usa.set_index('Province_State')
usa_demo = usa_demo.loc[['New York'],['Confirmed', 'Deaths', 'Recovered', 'Active']]
usa_demo = usa_demo.reset_index()
usa_demo[['Date']] = [[date[0]]]
usa_demo = usa_demo.set_index('Date')
usa_demo

,Province_State,Confirmed,Deaths,Recovered,Active
Date,,,,,
04-12-2020,New York,203020,10842,23887.0,192178.0


In [37]:
usa_list = usa_demo.drop(usa_demo.index[0])
for i in date:
    demo = pd.read_csv('../data/us_report/{}.csv'.format(i))
    demo = demo.set_index('Province_State')
    demo = demo.loc[['New York'],['Confirmed', 'Deaths', 'Recovered', 'Active']]
    demo = demo.reset_index()
    demo[['Date']] = [[i]]
    usa_list = pd.concat([usa_list, demo])
usa_list = usa_list.set_index('Date')
usa_list = usa_list.drop('Province_State',axis=1)
usa_list.head()

,Confirmed,Deaths,Recovered,Active
Date,,,,
04-12-2020,189033,9385,23887.0,179648.0
04-13-2020,195749,10058,23887.0,185691.0
04-14-2020,203020,10842,23887.0,192178.0
04-15-2020,214454,11617,23887.0,202837.0
04-16-2020,223691,14832,23887.0,208859.0


In [38]:
usa_list.tail()

,Confirmed,Deaths,Recovered,Active
Date,,,,
11-12-2020,545762,33975,81198.0,430589.0
11-13-2020,551163,33993,81390.0,435780.0
11-14-2020,556551,34010,81585.0,440956.0
11-15-2020,560200,34032,81788.0,444380.0
11-16-2020,563690,34054,81908.0,447728.0


这道题一开始就想到怎么循环并表，先拿了张表做基本功能测试，达到数据要求后再做循环，然后在并表环节想到建一个空表追加数据，最后对数据进行调整，想的蛮辛苦，也踩了不少坑，因疏忽其中一个表名没改过了，结果导出的数据都一样，看花眼一直看不出来，不过完成后很有成就感，收获满满！😏😏

### Ex2：实现join函数

请实现带有`how`参数的`join`函数

* 假设连接的两表无公共列
* 调用方式为 `join(df1, df2, how="left")`
* 给出测试样例

In [39]:
df1 = pd.DataFrame({'Name':['San Zhang', 'Si Li', 'Wu Wang'],
                    'Age':[20, 21 ,21],
                    'Class':['one', 'two', 'three']})
df2 = pd.DataFrame({'NameX':['XSan Zhang', 'XLi Si', 'XWu Wang'],
                    'AgeX':[220, 221 ,221],
                    'ClassX':['Xone', 'Xtwo', 'XThree']})
df1

,Name,Age,Class
0,San Zhang,20,one
1,Si Li,21,two
2,Wu Wang,21,three


In [40]:
df2

,NameX,AgeX,ClassX
0,XSan Zhang,220,Xone
1,XLi Si,221,Xtwo
2,XWu Wang,221,XThree


In [41]:
df1.join(df2, how='left')

,Name,Age,Class,NameX,AgeX,ClassX
0,San Zhang,20,one,XSan Zhang,220,Xone
1,Si Li,21,two,XLi Si,221,Xtwo
2,Wu Wang,21,three,XWu Wang,221,XThree


In [42]:
def join(s1, s2, how):
    aa, bb = [], []
    if how=='left':
        for x in s1.columns:
            aa.append(x)
        s1 = s1.T
        for i in s1.values.tolist():
            bb.append(i)
        for y in s2.columns:
            aa.append(y)
        s2 = s2.T
        for j in s2.values.tolist():
            bb.append(j)
        df = pd.DataFrame(data=bb, index=aa)
        return df.T
    else:
        return 0
join(df1, df2, how='left')

,Name,Age,Class,NameX,AgeX,ClassX
0,San Zhang,20,one,XSan Zhang,220,Xone
1,Si Li,21,two,XLi Si,221,Xtwo
2,Wu Wang,21,three,XWu Wang,221,XThree


费尽心思，只能做到这个地步了，这个只能算是对df1和df2这两个定制版吧。😆

休息休息，明天还要上班，今天已经花完所有脑细胞了，估计再想下去也是白搭，有充足时间做做练习挺好，不管会不会，总能活跃一下思维，明天去看看期中测试，没指望能做出来，就是学习学习。